In [ ]:
import pickle
import os
from dotenv import load_dotenv
import boto3


import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

from sklearn.pipeline import make_pipeline
import mlflow

SyntaxError: invalid syntax (2072295046.py, line 5)

In [6]:
# Load .env
load_dotenv()

# S3 client
s3 = boto3.client("s3")
bucket = os.getenv("S3_BUCKET_NAME")

print(f"Using bucket: {bucket}")
print(s3.list_objects(Bucket=bucket))

Using bucket: taxi-duration-artifacts
{'ResponseMetadata': {'RequestId': 'X317Y1ZK4EDZCZE9', 'HostId': 'g79eYAGAMKF6fHylOSNHaZG4x/lvfHexVjFbRdIJx1YKg5DM2SsZLeC33xzSSdFBruQmRL8WVfctiQdf4P7FAdimNeApRQHNKwdHWAr5+JI=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'g79eYAGAMKF6fHylOSNHaZG4x/lvfHexVjFbRdIJx1YKg5DM2SsZLeC33xzSSdFBruQmRL8WVfctiQdf4P7FAdimNeApRQHNKwdHWAr5+JI=', 'x-amz-request-id': 'X317Y1ZK4EDZCZE9', 'date': 'Fri, 03 Oct 2025 20:49:19 GMT', 'x-amz-bucket-region': 'us-east-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Marker': '', 'Name': 'taxi-duration-artifacts', 'Prefix': '', 'MaxKeys': 1000, 'EncodingType': 'url'}


In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("experiment-1-linear-models")

In [ ]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts


In [ ]:
df_train = read_dataframe('/workspaces/mlops-environment/02-training/data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('/workspaces/mlops-environment/02-training/data/green_tripdata_2025-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [ ]:
mlflow.autolog()
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

In [ ]:
from mlflow.tracking import MlflowClient

In [ ]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'b4d3bca8aa8e46a6b8257fe4541b1136'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)